# Closures

## Introduction

- As we know functions defined inside another function can access the outer (nonlocal) variables. Suppose consider the following example

  ```python

  def outer():

    x = 'python'

    def inner():

        print("{0} rocks!".format(x))

    return inner

  fn = outer()

  fn()

  ```

  In this code, varibale `x` in inner() refers to one in outer's scope and we call this nonlocal variable `x` as free variable. In this code when we call outer() function then it first a new scope gets created and in that scope variable `x` and function `inner` gets created. And here we are returning the function object. Once the `inner()` function object gets returned then scope of outer() function gets deleted. Then variable `x` also gets deleted. Now a new question is raised i.e if variable `x` gets deleted then how the inner() function access the variable `x` now. But it can access that variable `x` and prints te outputs as `python rocks!`. The reason behind this output is because of closures.

- A closure is a function object that remembers the values in enclosing scopes even if they are not in memory anymore. To eloborate this a closure occurs when a nested function (a function defined inside another function) remembers and uses variables from its enclosing scope, even after the outer function has finished executing. So simply closure is function and free variables. In above example closure is nothing but function `inner` and free variable `x`. 

- So what python actually doing is it actually creates an internal object called `cell` and that cell points to object refereing by the free varaible and these free variable in outer and inner refers to this cell. And object returned by the outer function is not just function object it is actually a closure. Since these closures remembering the free variables by using the indirect reference called `cell`, they can able to get those nonlocal variables eventhough nonlocal scope got exhausted.

- CLosures only form with nonlocal variables only. They don't form with global varaibles. To know the free variables and closures, we can use these syntax. Those are : `<function_object or closure>.__code__.cofreevars` to get free vriables and `<function_object or closure>.__closure__` to get the closure cell address and its reference object.

  **Ex**

  ```python

  def outer():

    x = "python"

    def inner():

      print(x)

    return inner

  fn = outer()
  fn()

  ```

  Here the value of `x` is shared between the two scopes. One is `outer` and another one is `closure`. Here the label of x is in two different scopes but always reference the same value. Python does this by creating the a `cell` an intermediary object which refers the actual value of `x` and `x` in two different scopes (`outer` and `inner`) refers to this `cell` object. When we requesting the value of the variable, python will double hop and gets the value. 


  <img src="_static\closure.png" alt="Closure Mechanism in Python" width="1200"/>

  From the image we can say closure as `function -> inner + extended scope x (cell)`

- So we can think closure as a function + an extended scope that contains the free variables. The free variables value is the object the cell points to -so that could change over time. Every time the function in the closure is called and the free variance gets referenced then python looks up the cell object and then return whatever the cell is pointing.



## Modifying the free variables

- Consider an the following code :

  ```python

  def counter():

    count = 0

    def inc():

        nonlocal count

        count += 1
        return count

    return inc
  
  fn = counter()
  fn()

  ```

  Here if you see the free variable is count. So closure contains the `inc()` function and free variable `count`. When counter counter() function gets executed, we get a closure as return value. At first cell points to the integer object `0`. Whenever that closure gets called the cell starts point to integer object `1`. If you again run the closure the cell now points to integer object `2`. But variable count in both counter() and inc() functions refer to same cell. Just the object referenced by cell gets changing here.


In [1]:
# Now lets see this in practice

def counter():

    count = 0

    def inc():
        
        nonlocal count

        count +=1

        return count
    return inc

fn = counter()

In [ ]:
# Now lets introspect the closure using fn

fn.__code__.co_freevars

# Here we can see the free variables in the closure which is count

('count',)

In [ ]:
# Now lets see the cell and what cell actually pointing to

fn.__closure__

# Here we can see the cell memory address and integer object memory address

(<cell at 0x000002B13010B400: int object at 0x000002B12A7D00D0>,)

In [ ]:
# As we know this integer is 0 and since 0 is singleton object, then it memory address is fixed.

hex(id(0))

# So if you compare both the memory address is same.

'0x2b12a7d00d0'

In [ ]:
# Now lets call the function

fn()

fn.__closure__

# If you see the address of integer object gets changed, now it should points to 1

(<cell at 0x000002B13010B400: int object at 0x000002B12A7D00F0>,)

In [ ]:
# Lets check the address of 1 as it is also a singleton object

hex(id(1))

# If you see both address are same

'0x2b12a7d00f0'

In [8]:
# Lets again call the function

fn()

fn.__closure__

(<cell at 0x000002B13010B400: int object at 0x000002B12A7D0110>,)

In [ ]:
hex(id(2))

# If you see in all the three cases cell object is same, the only thing changing is the object referenced by cell. This is how python modifies free variables.

'0x2b12a7d0110'

## Multiple Instances of Closure

- As we know, Everytime we run a function a new scope gets created. If the function generates a closure then a new closure also gets created everytime.

- Consider the same example provided above , if you executed the `fn1 = counter()` then a new scope and closure gets generated (that means new cell object gets created). If you again run the counter function as `fn2 = counter()` a new scope and closure gets created which is different from initial scope and clsoure.

In [10]:
def counter():

    count = 0

    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [11]:
fn1 = counter()

fn1.__closure__

(<cell at 0x000002B1301093F0: int object at 0x000002B12A7D00D0>,)

In [ ]:
fn2 = counter()

fn2.__closure__

# If you observe the cell addresses of bot closures, they are different. From this we can say everytime you run the function a new scope gets created
# If it generates closure, a new closure also gets created.

(<cell at 0x000002B130108CA0: int object at 0x000002B12A7D00D0>,)

## Shared Extended Scopes

- Consider the following example :

  ```python

  def outer():

    count = 0

    def inc1():
        nonlocal count

        count + = 1

        return count

    def inc2():
        nonlocal count

        count + = 1

        return count

    return inc1,inc2

  fn1,fn2 = outer()
  fn1()
  fn2()

  ```

- In this example if you see closure generated by the function inc1() and inc2() share same free variable which means both closures generates same cell object and free variable in these two closures points to same cell.

In [15]:
def outer():

    count = 0

    def inc1():
        nonlocal count

        count += 1

        return count

    def inc2():
        nonlocal count

        count += 1

        return count

    return inc1,inc2

fn1,fn2 = outer()

In [16]:
fn1.__closure__

(<cell at 0x000002B13010A260: int object at 0x000002B12A7D00D0>,)

In [17]:
fn2.__closure__

(<cell at 0x000002B13010A260: int object at 0x000002B12A7D00D0>,)

In [18]:
# If you see you share same cell object, so they will increment same count.

fn1()

1

In [19]:
fn2()

2

In [ ]:
fn1()

# If you see both are incrementing same count. This is how shared closures work.

3

In [25]:
# But we may think shared extended scopes are highly unusual which may not occur in real life scenario. But consider the following example

def adder(n):
    def inner(x):
        return x + n
    return inner

# Here n is free variable.

add_1 = adder(1) # This adder add 1 to each number you have provided

add_2 = adder(2) # This adder adds 2 to each number you have provided

add_3 = adder(3) # This adder adds 3 to each number you have provided

# Here we have got three different closures as we have called adder function three times.

add_1(10)

11

In [26]:
add_2(10)

12

In [27]:
add_3(10)

13

In [28]:
# But generally we don't like to hardcode these things if we have loops in python. So we can write above code as like this

adders = []
for n in range(1,4):

    adders.append(lambda x : x + n)

# Here we think all the three adders are append into list. But here we ave shared variable of n in all these three cases.

# At first n points to 1 and lambda function gets appended into adders list. Next n points to 2 since lambda function is not evaluated now,
# so n value in first lambda function also changed. At last all three lambda functions have same n which points to 3 only.

print(adders)

[<function <lambda> at 0x000002B12F10A950>, <function <lambda> at 0x000002B1302C7640>, <function <lambda> at 0x000002B1302C6D40>]


In [ ]:
print(adders[0](10))
print(adders[1](10))
print(adders[2](10))

# As we see all results same value. These problems might happen if we have shared varaibles. So we need to take care while dealing problems with shared closures or variables.


13
13
13


In [ ]:
# Here lambda is not a closure as n is global variable. Closure generated only with nonlocal variables.

adders[0].__closure__

# If you see nothing gets printed

In [ ]:
adders[0].__code__.co_freevars

# We can see that we got an empty tuple of free variables. From these we can say that clsoures are formed with nonlocal variables only.

()

**Note** : Similar to shared closures we have nested closures also which are also follows same concept. One example for nested closures is the following code.

```python

def incrementor(n):

    def inner(start):
        current = start

        def inc():
            nonlocal current

            current += n
            return current

        return inc
    
    return inner

```

`fn = incrementor(2)` -> `fn.__code__.co_freevars` -> ('n') where  n = 2

`inc_2 = fn(100)` -> `inc_2.__code__.co_freevars` -> ('current', 'n') where current = 100, n = 2

`inc_2()` -> 102

`inc_2()` -> 104